In [1]:
#import modules and set up environment 
import os
import sys
path = "../../src/"

sys.path.append(path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import jax.numpy as jnp
from jax import jit, grad, vmap, hessian, scipy, random

In [2]:
#import sgmcmc code 
import models.logistic_regression.logistic_regression as lr
import samplers.sgld as sgld
import samplers.sgldps as sgldps
import samplers.sgldcv as sgldcv
import samplers.sgldcvps as sgldcvps
import samplers.sgd as sgd

key = random.PRNGKey(0)

### Set up model and data

In [3]:
#load in the data
file_path = "../../data/synthetic/toy_lr_imbalance_train_synth.csv"
data = pd.read_csv(file_path)
dat_array = data.values[:]
x = dat_array[:, 1:]
y = dat_array[:,0]

sum(y)

988.0

In [4]:
#set up model parameters
N = x.shape[0]
dim = x.shape[1] 

#priors
mu_0 = np.zeros(dim) #prior mean
lambda_0 = 10.0*np.eye(dim)  #prior covariance matrix

In [5]:
#set up model-specific gradient functions
gradf_0 = lr.gradf_0
gradf_i_batch = lr.gradf_i_batch
post_var = lr.post_var

### Find mode using SGD

In [6]:
#hyperparameters for SGD
n_sgd = int(0.01*N)
Niter_sgd = 10**4
step_sgd = 1e-3

In [7]:
#draw theta_0 from the prior
theta_start = jnp.zeros(dim) #random starting value
print("Initalising SGD to find mode at theta_0:\n", theta_start)

Initalising SGD to find mode at theta_0:
 [0. 0. 0. 0. 0.]


In [8]:
theta_hat, samples_sgd, run_time = sgd.adam(key, gradf_0, gradf_i_batch, Niter_sgd, theta_start, x, y, n_sgd, step_sgd, replacement=True)

In [9]:
theta_hat

array([ 0.93656433, -1.30027044,  1.02401328, -1.81391406,  0.11790969])

In [10]:
# draw candidate thetas
n_candidate = 10
sigma_hat = post_var(theta_hat, x,y)[0]
theta_candidates = random.multivariate_normal(key=key, mean = theta_hat, cov= sigma_hat, shape=(n_candidate,))
theta_candidates

DeviceArray([[ 0.74375594, -1.6845198 ,  0.80323344, -1.9994667 ,
              -0.02570256],
             [ 0.19526291, -1.7324963 ,  1.219984  , -2.1950338 ,
               0.35618848],
             [ 0.6943455 , -1.5787483 ,  1.1151823 , -1.9177628 ,
              -0.09174893],
             [-1.2193822 , -2.0068364 ,  1.9670094 , -2.3089626 ,
               0.84404826],
             [ 0.5718819 , -1.3341489 ,  1.4023839 , -2.019978  ,
              -0.07550318],
             [ 2.4766343 , -0.9882709 ,  0.50295043, -2.0399265 ,
              -0.9024755 ],
             [ 1.5141598 , -1.0079905 ,  0.88360703, -1.6251979 ,
               0.07107104],
             [ 1.0993037 , -1.2664223 ,  1.1931986 , -1.4051132 ,
               0.4355286 ],
             [ 0.00617558, -1.603772  ,  1.4576167 , -1.8127699 ,
               0.3679529 ],
             [ 0.6201044 , -1.772526  ,  0.9849154 , -2.4414155 ,
              -0.2622505 ]], dtype=float32)

### Set up experiment 

In [11]:
batch_sizes = jnp.int32(jnp.linspace(0.01, 0.99, num=50)*N)

In [12]:
pseudo_var = dict()
reps = 1000

### SGLD gradients (with replacement)

In [13]:
pseudo_var['sgld-wr'] = np.zeros((n_candidate, batch_sizes.shape[0])) 
#sgld (wr) gradients
for i in tqdm(range(n_candidate), desc = "Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgld_gradients1 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgld.sgld_grad(subkey, theta_candidate, gradf_0, gradf_i_batch, x, y, n, replacement=True)
            sgld_gradients1[k] = grad
        pseudo_var['sgld-wr'][i,j] = np.sum(np.nanvar(sgld_gradients1, axis = 0))

Theta candidates tested: 100%|██████████| 10/10 [04:41<00:00, 28.17s/it]


### SGLD gradients (without replacement)

In [14]:
pseudo_var['sgld-wor'] = np.zeros((n_candidate, batch_sizes.shape[0])) 
#sgld (wor) gradients
for i in tqdm(range(n_candidate), desc = "Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgld_gradients2 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgld.sgld_grad(subkey, theta_candidate, gradf_0, gradf_i_batch, x, y, n, replacement=False)
            sgld_gradients2[k] = grad
        pseudo_var['sgld-wor'][i,j] = np.sum(np.nanvar(sgld_gradients2, axis = 0))

Theta candidates tested: 100%|██████████| 10/10 [12:50<00:00, 77.08s/it]


### SGLD-PS (exact probabilities)

In [15]:
#sgld-ps (exact) gradients
pseudo_var['sgld-ps-exact'] = np.zeros((n_candidate, batch_sizes.shape[0])) 

In [16]:
for i in tqdm(range(n_candidate), desc = "Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    probs1 = sgldps.exact_probs(theta_candidate, gradf_i_batch, x, y)
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgldps_gradients1 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgldps.sgldps_grad(subkey, theta_candidate, gradf_0, gradf_i_batch, probs1, x, y, n)
            sgldps_gradients1[k] = grad
        pseudo_var['sgld-ps-exact'][i,j] = np.sum(np.nanvar(sgldps_gradients1, axis = 0))

Theta candidates tested: 100%|██████████| 10/10 [13:33<00:00, 81.36s/it]


### SGLD-PS (approximate probabilities)

In [17]:
#sgld-ps (approx) gradients
f_i_grad_list = sgldps.ps_preliminaries(theta_hat, gradf_i_batch, x, y)
probs2 = sgldps.approx_probs(theta_hat, f_i_grad_list)
pseudo_var['sgld-ps-approx'] = np.zeros((n_candidate, batch_sizes.shape[0])) 

In [18]:
for i in tqdm(range(n_candidate), desc = " Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgldps_gradients2 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgldps.sgldps_grad(subkey, theta_candidate, gradf_0, gradf_i_batch, probs2, x, y, n)
            sgldps_gradients2[k] = grad
        pseudo_var['sgld-ps-approx'][i, j] = np.sum(np.nanvar(sgldps_gradients2, axis = 0))

 Theta candidates tested: 100%|██████████| 10/10 [12:57<00:00, 77.79s/it]


### SGLD-CV gradients (with replacement)

In [19]:
gradf_0_hat, grad_full_hat, f_i_grad_list = sgldcv.cv_preliminaries(theta_hat, gradf_0, gradf_i_batch, x, y)

In [20]:
pseudo_var['sgld-cv-wr'] = np.zeros((n_candidate, batch_sizes.shape[0])) 

#sgldcv gradients
for i in tqdm(range(n_candidate), desc = "Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgldcv_gradients1 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgldcv.sgld_cv_grad(subkey, theta_candidate, theta_hat, gradf_0, gradf_i_batch, grad_full_hat, f_i_grad_list, x, y, n, replacement=True)
            sgldcv_gradients1[k] = grad
        pseudo_var['sgld-cv-wr'][i, j] = np.sum(np.nanvar(sgldcv_gradients1, axis = 0))

Theta candidates tested: 100%|██████████| 10/10 [08:33<00:00, 51.37s/it]


### SGLD-CV gradients (without replacement)

In [21]:
pseudo_var['sgld-cv-wor'] = np.zeros((n_candidate, batch_sizes.shape[0])) 

#sgldcv gradients
for i in tqdm(range(n_candidate), desc = "Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgldcv_gradients2 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgldcv.sgld_cv_grad(subkey, theta_candidate, theta_hat, gradf_0, gradf_i_batch, grad_full_hat, f_i_grad_list, x, y, n, replacement=False)
            sgldcv_gradients2[k] = grad
        pseudo_var['sgld-cv-wor'][i, j] = np.sum(np.nanvar(sgldcv_gradients2, axis = 0))

Theta candidates tested: 100%|██████████| 10/10 [17:24<00:00, 104.48s/it]


### SGLD-CV-PS (exact probabilities) gradients

In [22]:
cov_mat = sgldcvps.cvps_preliminaries(theta_hat, gradf_0, gradf_i_batch, post_var, x, y)[0]
f_i_hess_list = sgldcvps.cvps_preliminaries(theta_hat, gradf_0, gradf_i_batch, post_var, x, y)[4]

In [23]:
#sgld-cv-ps (exact) gradients
pseudo_var['sgld-cv-exact'] = np.zeros((n_candidate, batch_sizes.shape[0])) 

for i in tqdm(range(n_candidate), desc = " Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    probs3 = sgldcvps.exact_probs(theta_candidate, theta_hat, gradf_i_batch, f_i_grad_list, x, y)
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgldcvps_gradients1 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgldcvps.sgldcv_ps_grad(subkey, theta_candidate, theta_hat, gradf_0, gradf_i_batch, grad_full_hat, f_i_grad_list, probs3, x, y, n)
            sgldcvps_gradients1[k] = grad
        pseudo_var['sgld-cv-exact'][i,j] = np.sum(np.nanvar(sgldcvps_gradients1, axis = 0))

 Theta candidates tested: 100%|██████████| 10/10 [17:36<00:00, 105.64s/it]


### SGLD-CV-PS (approximate probabilities) gradients

In [24]:
#sgldps (approx) gradients
probs4 = sgldcvps.approx_probs_fro(theta_hat, f_i_hess_list)

In [25]:
pseudo_var['sgld-cv-approx'] = np.zeros((n_candidate, batch_sizes.shape[0])) 
for i in tqdm(range(n_candidate), desc = "Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgldcvps_gradients2 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgldcvps.sgldcv_ps_grad(subkey, theta_candidate, theta_hat, gradf_0, gradf_i_batch, grad_full_hat, f_i_grad_list, probs4, x, y, n)
            sgldcvps_gradients2[k] = grad
        pseudo_var['sgld-cv-approx'][i,j] = np.sum(np.nanvar(sgldcvps_gradients2, axis = 0))

Theta candidates tested: 100%|██████████| 10/10 [17:26<00:00, 104.62s/it]


### Save results 

In [26]:
plot_data = []
for key in pseudo_var:
    smooth_path = pseudo_var[key].mean(axis = 0)
    plot_data.append(smooth_path)
    path_deviation = 2*pseudo_var[key].std(axis = 0)
    under_line = (smooth_path-path_deviation)
    plot_data.append(under_line)
    over_line = (smooth_path+path_deviation)
    plot_data.append(over_line) 
    
df = pd.DataFrame(plot_data).T
df['proportion'] = np.linspace(0.01, 0.99, num = 50)

In [27]:
path_out = "./out/toy_lrib_gradient_comp.csv"
df.to_csv(path_out, index = False) #save csv